In [115]:
import streamlit as st
import pandas as pd
import plotly.express as px 
import plotly.graph_objects as go
import numpy as np



DATA_URL = 'get_around_pricing_project.csv'

def load_data(): 
    data = pd.read_csv(DATA_URL)
    return data

data = load_data()

In [116]:
def describe_df(df):
    print('number of rows:',len(df))
    print('Display of dataset:')
    display(df.head())
    print('Basic statistics:')
    display(df.describe(include="all"))
    print('Pourcentage of missing values:')
    display(df.isnull().sum()/len(df)*100)
describe_df(data)

number of rows: 4843
Display of dataset:


,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


Basic statistics:


,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
count,4843.000000,4843,4.843000e+03,4843.00000,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843.000000
unique,NaN,28,NaN,NaN,4,10,8,2,2,2,2,2,2,2,NaN
top,NaN,Citroën,NaN,NaN,diesel,black,estate,True,True,False,False,False,False,True,NaN
freq,NaN,969,NaN,NaN,4641,1633,1606,2662,3839,3865,3881,2613,3674,4514,NaN
mean,2421.000000,NaN,1.409628e+05,128.98823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.214536
std,1398.198007,NaN,6.019674e+04,38.99336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.568268
min,0.000000,NaN,-6.400000e+01,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000
25%,1210.500000,NaN,1.029135e+05,100.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.000000
50%,2421.000000,NaN,1.410800e+05,120.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.000000
75%,3631.500000,NaN,1.751955e+05,135.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.000000


Pourcentage of missing values:


Unnamed: 0                   0.0
model_key                    0.0
mileage                      0.0
engine_power                 0.0
fuel                         0.0
paint_color                  0.0
car_type                     0.0
private_parking_available    0.0
has_gps                      0.0
has_air_conditioning         0.0
automatic_car                0.0
has_getaround_connect        0.0
has_speed_regulator          0.0
winter_tires                 0.0
rental_price_per_day         0.0
dtype: float64

In [117]:
data = data.iloc[:,1:]
target = 'rental_price_per_day'

num_features = [c for c in data.columns if c != target]

def display_distribution(c):
    fig = px.scatter(data, c)
    fig.update_layout(width=700,height=500)
    fig.show()

# for c in num_features:
#     display_distribution(c)

In [118]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import joblib

In [119]:
data.head()

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [120]:
data['model_key'].value_counts()

model_key
Citroën        969
Renault        916
BMW            827
Peugeot        642
Audi           526
Nissan         275
Mitsubishi     231
Mercedes        97
Volkswagen      65
Toyota          53
SEAT            46
Subaru          44
PGO             33
Ferrari         33
Opel            33
Maserati        18
Suzuki           8
Porsche          6
Ford             5
KIA Motors       3
Alfa Romeo       3
Fiat             2
Lexus            2
Lamborghini      2
Mazda            1
Honda            1
Mini             1
Yamaha           1
Name: count, dtype: int64

In [121]:
model_counts = data['model_key'].value_counts()
data['model_key'] = data['model_key'].apply(lambda x: 'rare' if model_counts[x] < 10 else x)

In [122]:
data.head()

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [124]:
target = 'rental_price_per_day'
features = [c for c in data.columns if c != target]

# Define categorical and numerical features
categorical_features = [
    'model_key', 'fuel', 'paint_color', 'car_type', 'private_parking_available', 
    'has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect', 
    'has_speed_regulator', 'winter_tires'
]

numerical_features = ['mileage', 'engine_power']

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(drop="first")

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [125]:
a=False
if a:
    # Define models
    regressors = {
        'Random Forest': RandomForestRegressor(),
        'Gradient Boosting': GradientBoostingRegressor(),
        'XGBoost': XGBRegressor()
    }
    params_rf = {
        'regressor__n_estimators': [100, 200],
        'regressor__max_depth': [10, 20],
    }

    params_gb = {
        'regressor__n_estimators': [100, 200],
        'regressor__learning_rate': [0.1, 0.05],
        'regressor__max_depth': [3, 5]
    }

    params_xgb = {
        'regressor__n_estimators': [100, 200],
        'regressor__learning_rate': [0.1, 0.05],
        'regressor__max_depth': [3, 5]
    }

    param_grids = {
        'Random Forest': params_rf,
        'Gradient Boosting': params_gb,
        'XGBoost': params_xgb
    }

    # Split data into train and test sets
    X = data[features]
    y = data[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Evaluate each model using GridSearchCV
    results = {}
    for name, model in regressors.items():
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                ('regressor', model)])
        grid_search = GridSearchCV(pipeline, param_grid=param_grids[name], cv=5, scoring='neg_mean_absolute_error',verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results[name] = {'MAE': mae, 'MSE': mse, 'R2': r2}

    results_df = pd.DataFrame(results).T
    print(results_df)


In [126]:
regressors = {
    'XGBoost': XGBRegressor()
}

params_xgb = {
    'regressor__n_estimators': [500],
    'regressor__learning_rate': [0.0443],
    'regressor__max_depth': [5]
}
param_grids = {
    'XGBoost': params_xgb
}

# Split data into train and test sets
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to find best parameters
def find_best_params(regressors, param_grids, X_train, y_train, X_test, y_test):
    results = {}
    best_params = {}
    for name, model in regressors.items():
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('regressor', model)])
        grid_search = GridSearchCV(pipeline, param_grid=param_grids[name], cv=5, scoring='r2', verbose=0)
        # grid_search = GridSearchCV(pipeline, param_grid=param_grids[name], cv=5, scoring='neg_mean_absolute_error', verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_

        joblib.dump(best_model, f'best_model_{name}.pkl')

        y_pred = best_model.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results[name] = {'MAE': mae, 'MSE': mse, 'R2': r2}
        best_params[name] = grid_search.best_params_
    
    return results, best_params

# Find best parameters and results
results, best_params = find_best_params(regressors, param_grids, X_train, y_train, X_test, y_test)

# Display results
results_df = pd.DataFrame(results).T
print(results_df)

# Display best parameters
print("Best Parameters:")
for name, params in best_params.items():
    print(f"{name}: {params}")


              MAE         MSE        R2
XGBoost  10.37208  259.220416  0.753879
Best Parameters:
XGBoost: {'regressor__learning_rate': 0.0443, 'regressor__max_depth': 5, 'regressor__n_estimators': 500}


In [146]:
loaded_model = joblib.load('best_model_XGBoost.pkl')
random_row = data.sample(n=1, random_state=0)
actual_value = random_row[target].values[0]
new_data = random_row.drop(columns=[target])
new_predictions = loaded_model.predict(new_data)
print(f"Prediction for the selected row: {new_predictions[0]:.2f} $/day")
print(f"Actual value for the selected row: {actual_value} $/day")


Prediction for the selected row: 95.14 $/day
Actual value for the selected row: 97 $/day
